In [68]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

# Task 2: Merging Datasets with Transitive Properties

In [69]:
import pandas as pd

# Load datasets
transactions = pd.read_csv("customer_transactions_augmented.csv")
social_profiles = pd.read_csv("customer_social_profiles.csv")
id_mapping = pd.read_csv("id_mapping.csv")

# Merge customer_id_new into transactions before any operations
transactions = transactions.merge(id_mapping, on="customer_id_legacy", how="left")

# Merge transactions with social profiles using mapped IDs
merged_data = transactions.merge(social_profiles, on="customer_id_new", how="left")

# Handle conflicts: Aggregate by customer_id_new
merged_data = merged_data.groupby('customer_id_new').agg({
    'purchase_amount': 'sum',  # Total amount spent
    'purchase_interest_score': 'mean',  # Average purchase interest score
    'engagement_score': 'mean'  # Average engagement score
}).reset_index()

In [70]:
# -------------------------------
# ✅ Moving Average per Customer
# -------------------------------
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])
transactions = transactions.sort_values(['customer_id_new', 'purchase_date'])

transactions['moving_avg_transaction'] = transactions.groupby('customer_id_new')['purchase_amount'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)

# Merge moving average back to merged_data
avg_spending = transactions.groupby('customer_id_new')['moving_avg_transaction'].mean().reset_index()
merged_data = merged_data.merge(avg_spending, on="customer_id_new", how="left")

In [71]:
# -------------------------------
# ✅ Time-based Monthly Aggregation
# -------------------------------
transactions['purchase_month'] = transactions['purchase_date'].dt.to_period('M')

monthly_spending = transactions.groupby(['customer_id_new', 'purchase_month'])['purchase_amount'].sum().reset_index()
monthly_spending = monthly_spending.groupby('customer_id_new')['purchase_amount'].mean().reset_index()

# Merge monthly spending into merged_data
merged_data = merged_data.merge(monthly_spending, on="customer_id_new", how="left", suffixes=('', '_monthly'))

In [72]:
# -------------------------------
# ✅ Simulated Text Data for TF-IDF
# -------------------------------
def generate_text_features(row):
    text_features = []

    # Convert numerical scores into categorical labels
    if row['engagement_score'] > 0.7:
        text_features.append("high_engagement")
    elif row['engagement_score'] > 0.3:
        text_features.append("medium_engagement")
    else:
        text_features.append("low_engagement")

    if row['purchase_interest_score'] > 0.7:
        text_features.append("high_interest")
    elif row['purchase_interest_score'] > 0.3:
        text_features.append("medium_interest")
    else:
        text_features.append("low_interest")

    return " ".join(text_features)

# Apply text conversion
merged_data['text_features'] = merged_data.apply(generate_text_features, axis=1)

# Apply TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(merged_data['text_features'])

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Merge TF-IDF features back to merged_data
merged_data = pd.concat([merged_data, tfidf_df], axis=1)

In [73]:
# -------------------------------
# ✅ Customer Engagement Score
# -------------------------------
merged_data['customer_engagement_score'] = (
    0.5 * merged_data['purchase_amount'] +
    0.3 * merged_data['purchase_interest_score'] +
    0.2 * merged_data['engagement_score']
)

In [74]:
# Save the final dataset
merged_data.to_csv("final_customer_data_group_12.csv", index=False)
print("Final dataset saved as final_customer_data_group_12.csv")

Final dataset saved as final_customer_data_group_12.csv
